In [ ]:
!git clone https://personalaccesstoken@github.com/username/reponame.git

Cloning into 'reponame'...
fatal: could not read Password for 'https://personalaccesstoken@github.com': No such device or address


In [ ]:
!git clone https://github.com/jairotunior/dashboard_tutorial.git

Cloning into 'dashboard_tutorial'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 136 (delta 37), reused 127 (delta 28), pack-reused 0
Receiving objects: 100% (136/136), 683.72 KiB | 5.47 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [ ]:
!ls

dashboard_tutorial  sample_data


In [ ]:
import os
os.chdir('dashboard_tutorial/')

In [ ]:
!ls

dashboards	    datasets   requirements.txt  screenshots
dashboard_tutorial  README.md  run_dashboard.py


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.2/149.2 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
run_dashboard.py

In [ ]:
import os
import logging
from pathlib import Path
from dashboard_tutorial.sources import FREDSource, QuandlSource, FileSource
from dashboard_tutorial.transformers import FractionalDifferentiationEW, FractionalDifferentiationFFD, Differentiation, PercentageChange
from dashboard_tutorial.managers import ManagerDashboard
from dashboard_tutorial.views import DashboardView

if __name__ == '__main__':
    # Config Logging
    logging.basicConfig(format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', level=logging.INFO)

    BASE_DIR = Path(__file__).resolve().parent

    ANALYSIS_PATH = os.path.join(BASE_DIR, 'dashboards')

    if not os.path.exists(ANALYSIS_PATH):
        logging.info("[+] Se ha creado directorio de archivos.")
        os.mkdir(ANALYSIS_PATH)

    # Define Manager
    manager = ManagerDashboard(path=ANALYSIS_PATH)

    # **************** Register Transformers ***************************
    differentiation_transform = Differentiation(units_show='Returns')
    fractional_diff_transform = FractionalDifferentiationEW(units_show='Fractional Return')
    fractional_diff_ffd_transform = FractionalDifferentiationFFD(units_show='FFD Return')
    percentage_change_transform = PercentageChange(units_show='Percentage Change')
    percentage_change_from_year_transform = PercentageChange(name="Percentage Change from Year Ago", units_show='Percentage Change from Year Ago', periods=12)

    manager.transformers.register(fractional_diff_transform)
    manager.transformers.register(fractional_diff_ffd_transform)
    manager.transformers.register(differentiation_transform)
    manager.transformers.register(percentage_change_transform)
    manager.transformers.register(percentage_change_from_year_transform)


    # ******************* Create Source Managers ******************************
    fred_credentials = os.path.join(BASE_DIR, "api_fred.txt")
    fred_source = FREDSource(fred_credentials=fred_credentials)

    quandl_api_key = 'API_KEY'
    quandl_source = QuandlSource(api_key=quandl_api_key)

    # File source
    file_source_dir = os.path.join(BASE_DIR, "datasets", 'yields')
    file_source = FileSource(dir=file_source_dir)

    manager.sources.register(fred_source)
    manager.sources.register(quandl_source)
    manager.sources.register(file_source)

    manager.load()

    # ******************************************** Create Figures *************************************
    logging.info("[+] Render dashboards...")

    dashboard_view = DashboardView(title="Dashboard", manager=manager)

    dashboard_view.show()



NameError: ignored